# Contents
1. ### ABC XYZ analysis description
2. ### Data preparation
3. ### Analysis
4. ### Result

## Data preparation

### Importing librarires

In [557]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
import copy
import plotly.graph_objects as go
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl', offline=True)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/ecommerce-data/Sales target.csv
/kaggle/input/ecommerce-data/List of Orders.csv
/kaggle/input/ecommerce-data/Order Details.csv


### Reading files

In [558]:
orderdetails = pd.read_csv('../input/ecommerce-data/Order Details.csv')
orderlist = pd.read_csv('../input/ecommerce-data/List of Orders.csv')
target = pd.read_csv('../input/ecommerce-data/Sales target.csv')

### Looking at dataset with order details

In [559]:
orderdetails.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases
1,B-25601,66.0,-12.0,5,Clothing,Stole
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games
4,B-25602,168.0,-111.0,2,Electronics,Phones


### Checking null and missning values

In [560]:
orderdetails.isnull().sum()

Order ID        0
Amount          0
Profit          0
Quantity        0
Category        0
Sub-Category    0
dtype: int64

### Reading some dataset descriptions

In [561]:
print('Orderdetails info')
print(orderdetails.info())
print('Orderdetails describe')
print(orderdetails.describe())

Orderdetails info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1500 entries, 0 to 1499
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      1500 non-null   object 
 1   Amount        1500 non-null   float64
 2   Profit        1500 non-null   float64
 3   Quantity      1500 non-null   int64  
 4   Category      1500 non-null   object 
 5   Sub-Category  1500 non-null   object 
dtypes: float64(2), int64(1), object(3)
memory usage: 70.4+ KB
None
Orderdetails describe
            Amount       Profit     Quantity
count  1500.000000  1500.000000  1500.000000
mean    287.668000    15.970000     3.743333
std     461.050488   169.140565     2.184942
min       4.000000 -1981.000000     1.000000
25%      45.000000    -9.250000     2.000000
50%     118.000000     9.000000     3.000000
75%     322.000000    38.000000     5.000000
max    5729.000000  1698.000000    14.000000


### Looking at the dataset with orders

In [562]:
orderlist.head()

,Order ID,Order Date,CustomerName,State,City
0,B-25601,01-04-2018,Bharat,Gujarat,Ahmedabad
1,B-25602,01-04-2018,Pearl,Maharashtra,Pune
2,B-25603,03-04-2018,Jahan,Madhya Pradesh,Bhopal
3,B-25604,03-04-2018,Divsha,Rajasthan,Jaipur
4,B-25605,05-04-2018,Kasheen,West Bengal,Kolkata


### Reading some second dataset descriptions

In [563]:
print('Orderlist info')
print(orderlist.info())
print('Orderlist describe')
print(orderlist.describe())

Orderlist info
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Order ID      500 non-null    object
 1   Order Date    500 non-null    object
 2   CustomerName  500 non-null    object
 3   State         500 non-null    object
 4   City          500 non-null    object
dtypes: object(5)
memory usage: 22.0+ KB
None
Orderlist describe
       Order ID  Order Date CustomerName           State    City
count       500         500          500             500     500
unique      500         307          332              19      24
top     B-25616  24-11-2018       Shreya  Madhya Pradesh  Indore
freq          1           7            6             101      76


### Checking missing values in second dataset

In [564]:
orderlist.isnull().sum()

Order ID        60
Order Date      60
CustomerName    60
State           60
City            60
dtype: int64

### Creating heatmap of the dataset to check if there are 60 rows with empty sells or there are empty rows. As we can see it is empty rows.

In [565]:
cols = orderlist.columns
df=orderlist[cols].isnull().replace({True:1, False:0})
fig = px.imshow(df,x=df.columns, y=orderlist.index, labels=dict(x="Column", y="Row", color="Missing indicator"))
fig.update_layout(title = 'Missing values map (yellow for missing value cells and blue for filled cells)')
fig.show()

### Dropping empty rows from second dataset

In [566]:
orderlist = orderlist.dropna()

## Analysis

### Joining 2 datasets for analysis

In [567]:
orders = orderdetails.merge(orderlist[['Order ID','Order Date']], left_on='Order ID',right_on='Order ID', how='inner')
orders.head()

,Order ID,Amount,Profit,Quantity,Category,Sub-Category,Order Date
0,B-25601,1275.0,-1148.0,7,Furniture,Bookcases,01-04-2018
1,B-25601,66.0,-12.0,5,Clothing,Stole,01-04-2018
2,B-25601,8.0,-2.0,3,Clothing,Hankerchief,01-04-2018
3,B-25601,80.0,-56.0,4,Electronics,Electronic Games,01-04-2018
4,B-25602,168.0,-111.0,2,Electronics,Phones,01-04-2018


### Creating a copy of new dataset

In [568]:
orders_copy = copy.deepcopy(orders)

### Creating row with month and year of order to get know month good flow

In [569]:
orders_copy['Date'] = orders_copy['Order Date'].apply(lambda x:x[6::]+'-'+x[3:5])

### Creating dataset with information about monthly profit by goods' sub-categories

In [570]:
orderssub = orders_copy.groupby(['Sub-Category', 'Date']).Profit.sum().unstack().reset_index()
orderssub.head()

Date,Sub-Category,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03
0,Accessories,-55.0,-66.0,-89.0,30.0,-29.0,265.0,-604.0,466.0,320.0,997.0,1401.0,923.0
1,Bookcases,-1014.0,-75.0,NaN,-33.0,986.0,612.0,-1730.0,1704.0,292.0,2458.0,1101.0,587.0
2,Chairs,-444.0,-555.0,-426.0,-176.0,-193.0,-991.0,418.0,1328.0,232.0,665.0,258.0,461.0
3,Electronic Games,-326.0,-1481.0,-1220.0,-80.0,-372.0,-471.0,399.0,847.0,723.0,511.0,212.0,22.0
4,Furnishings,-103.0,-98.0,16.0,-248.0,-350.0,-115.0,110.0,640.0,112.0,161.0,382.0,337.0


### Creating column with full all profits for a known period by sub-categories

In [571]:
orderssub['Profit Summary'] = orderssub.iloc[:,1:14].sum(axis=1)

In [572]:
orderssub

Date,Sub-Category,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,Profit Summary
0,Accessories,-55.0,-66.0,-89.0,30.0,-29.0,265.0,-604.0,466.0,320.0,997.0,1401.0,923.0,3559.0
1,Bookcases,-1014.0,-75.0,NaN,-33.0,986.0,612.0,-1730.0,1704.0,292.0,2458.0,1101.0,587.0,4888.0
2,Chairs,-444.0,-555.0,-426.0,-176.0,-193.0,-991.0,418.0,1328.0,232.0,665.0,258.0,461.0,577.0
3,Electronic Games,-326.0,-1481.0,-1220.0,-80.0,-372.0,-471.0,399.0,847.0,723.0,511.0,212.0,22.0,-1236.0
4,Furnishings,-103.0,-98.0,16.0,-248.0,-350.0,-115.0,110.0,640.0,112.0,161.0,382.0,337.0,844.0
5,Hankerchief,-137.0,-25.0,-290.0,-75.0,-1.0,-33.0,374.0,608.0,340.0,386.0,381.0,570.0,2098.0
6,Kurti,-47.0,NaN,-92.0,NaN,-119.0,-95.0,0.0,200.0,177.0,65.0,2.0,90.0,181.0
7,Leggings,43.0,-19.0,-13.0,-56.0,-42.0,-10.0,27.0,73.0,41.0,26.0,33.0,157.0,260.0
8,Phones,327.0,-608.0,-163.0,-1847.0,-205.0,-62.0,1201.0,748.0,1628.0,901.0,-352.0,639.0,2207.0
9,Printers,-297.0,-368.0,-606.0,264.0,-942.0,-642.0,1265.0,1877.0,525.0,2376.0,666.0,1846.0,5964.0


### Get know what sub-categories cause losses. These is Electronic games and Tables

In [573]:
orderssub.loc[orderssub['Profit Summary'] < 0]['Sub-Category'].unique()

array(['Electronic Games', 'Tables'], dtype=object)

In [574]:
fig = go.Figure(data=[go.Bar(x=orderssub[orderssub['Profit Summary'] < 0]['Sub-Category'], y=orderssub[orderssub['Profit Summary'] < 0]['Profit Summary'])])
# Customize aspect
fig.update_traces(marker_color='rgb(255,0,17)', marker_line_color='rgb(173,0,0)',
                  marker_line_width=1.5, opacity=0.6)
fig.update_layout(title_text='Non profitable sub-categories: Loss')
fig.show()

In [575]:
orderssub[orderssub['Profit Summary'] < 0]

Date,Sub-Category,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,Profit Summary
3,Electronic Games,-326.0,-1481.0,-1220.0,-80.0,-372.0,-471.0,399.0,847.0,723.0,511.0,212.0,22.0,-1236.0
15,Tables,-1864.0,-66.0,-446.0,NaN,NaN,-1974.0,-114.0,273.0,-449.0,NaN,427.0,202.0,-4011.0


### Drop sub-categories which cause losses

In [576]:
orderssub = orderssub.loc[orderssub['Profit Summary'] > 0]
orderssub = orderssub.reset_index(drop=True)

### Creating new columns. Profit share is shows what contribution does the category make to the formation of profits, Cumulative share is cumulative total

In [577]:
orderssub['Profit share'] = orderssub['Profit Summary']/orderssub['Profit Summary'].values.sum()*100
orderssub = orderssub.sort_values('Profit share', ascending = False)
orderssub['Cumulative share'] = orderssub['Profit share'].cumsum()
orderssub

Date,Sub-Category,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,Profit Summary,Profit share,Cumulative share
8,Printers,-297.0,-368.0,-606.0,264.0,-942.0,-642.0,1265.0,1877.0,525.0,2376.0,666.0,1846.0,5964.0,20.423259,20.423259
1,Bookcases,-1014.0,-75.0,NaN,-33.0,986.0,612.0,-1730.0,1704.0,292.0,2458.0,1101.0,587.0,4888.0,16.738580,37.161838
0,Accessories,-55.0,-66.0,-89.0,30.0,-29.0,265.0,-604.0,466.0,320.0,997.0,1401.0,923.0,3559.0,12.187521,49.349360
14,Trousers,225.0,49.0,109.0,114.0,373.0,-815.0,103.0,865.0,81.0,175.0,93.0,1475.0,2847.0,9.749332,59.098692
12,Stole,-14.0,-4.0,-31.0,-151.0,-135.0,-176.0,321.0,520.0,407.0,699.0,308.0,815.0,2559.0,8.763098,67.861790
7,Phones,327.0,-608.0,-163.0,-1847.0,-205.0,-62.0,1201.0,748.0,1628.0,901.0,-352.0,639.0,2207.0,7.557702,75.419492
4,Hankerchief,-137.0,-25.0,-290.0,-75.0,-1.0,-33.0,374.0,608.0,340.0,386.0,381.0,570.0,2098.0,7.184439,82.603931
13,T-shirt,-5.0,77.0,25.0,-26.0,100.0,-71.0,69.0,200.0,232.0,281.0,256.0,362.0,1500.0,5.136634,87.740566
10,Shirt,21.0,1.0,-216.0,88.0,30.0,-29.0,8.0,130.0,240.0,199.0,492.0,167.0,1131.0,3.873022,91.613588
3,Furnishings,-103.0,-98.0,16.0,-248.0,-350.0,-115.0,110.0,640.0,112.0,161.0,382.0,337.0,844.0,2.890213,94.503801


In [578]:
fig = px.bar(orderssub, y='Profit Summary', x='Sub-Category', title='Profit by sub-category', color='Cumulative share')
fig.show()

### Defining ABC categories. A is goods which give 80% of profit, B is 15%, C is 5%.

In [579]:
orderssub['ABC'] = 0
orderssub.loc[(orderssub['Cumulative share'] <= 83), 'ABC'] = 'A'
orderssub.loc[(orderssub['Cumulative share'] <= 95)&(orderssub['Cumulative share'] >= 83), 'ABC'] = 'B'
orderssub.loc[(orderssub['Cumulative share'] >= 95), 'ABC'] = 'C'
orderssub

Date,Sub-Category,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,Profit Summary,Profit share,Cumulative share,ABC
8,Printers,-297.0,-368.0,-606.0,264.0,-942.0,-642.0,1265.0,1877.0,525.0,2376.0,666.0,1846.0,5964.0,20.423259,20.423259,A
1,Bookcases,-1014.0,-75.0,NaN,-33.0,986.0,612.0,-1730.0,1704.0,292.0,2458.0,1101.0,587.0,4888.0,16.738580,37.161838,A
0,Accessories,-55.0,-66.0,-89.0,30.0,-29.0,265.0,-604.0,466.0,320.0,997.0,1401.0,923.0,3559.0,12.187521,49.349360,A
14,Trousers,225.0,49.0,109.0,114.0,373.0,-815.0,103.0,865.0,81.0,175.0,93.0,1475.0,2847.0,9.749332,59.098692,A
12,Stole,-14.0,-4.0,-31.0,-151.0,-135.0,-176.0,321.0,520.0,407.0,699.0,308.0,815.0,2559.0,8.763098,67.861790,A
7,Phones,327.0,-608.0,-163.0,-1847.0,-205.0,-62.0,1201.0,748.0,1628.0,901.0,-352.0,639.0,2207.0,7.557702,75.419492,A
4,Hankerchief,-137.0,-25.0,-290.0,-75.0,-1.0,-33.0,374.0,608.0,340.0,386.0,381.0,570.0,2098.0,7.184439,82.603931,A
13,T-shirt,-5.0,77.0,25.0,-26.0,100.0,-71.0,69.0,200.0,232.0,281.0,256.0,362.0,1500.0,5.136634,87.740566,B
10,Shirt,21.0,1.0,-216.0,88.0,30.0,-29.0,8.0,130.0,240.0,199.0,492.0,167.0,1131.0,3.873022,91.613588,B
3,Furnishings,-103.0,-98.0,16.0,-248.0,-350.0,-115.0,110.0,640.0,112.0,161.0,382.0,337.0,844.0,2.890213,94.503801,B


In [580]:
fig = px.pie(orderssub.groupby('ABC')['Profit Summary'].sum(), values='Profit Summary', names = orderssub.groupby('ABC')['Profit Summary'].sum().index, title = 'Profit by ABC')
fig.show()

### Creating dataset with information about sales by sub-category.

In [581]:
ordersubb = orders_copy.groupby(['Sub-Category', 'Date']).Amount.sum().unstack().reset_index()
ordersubb.head()

Date,Sub-Category,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03
0,Accessories,68.0,850.0,349.0,187.0,759.0,1470.0,1339.0,1515.0,2588.0,4871.0,3967.0,3765.0
1,Bookcases,3374.0,1455.0,NaN,1489.0,8004.0,2989.0,3878.0,5274.0,2773.0,15564.0,5507.0,6554.0
2,Chairs,2251.0,3404.0,1415.0,840.0,570.0,1746.0,1854.0,5930.0,2765.0,3923.0,1764.0,7760.0
3,Electronic Games,1728.0,3859.0,1953.0,328.0,4251.0,975.0,2935.0,5481.0,4791.0,7503.0,1903.0,3461.0
4,Furnishings,1132.0,751.0,673.0,1154.0,964.0,355.0,402.0,1974.0,542.0,1770.0,2883.0,884.0


### Dropping non-profit sub-categories (information is above)

In [582]:
ordersubb = ordersubb[ordersubb['Sub-Category'].isin(orderssub.loc[orderssub['Profit Summary'] > 0]['Sub-Category'].unique())]

### Creating new column with information about gross given by sub-catgory for all known period

In [583]:
ordersubb['Summary'] = ordersubb.iloc[:,1:14].sum(axis=1)

In [584]:
ordersubb = ordersubb.sort_values('Summary', ascending = False)
ordersubb

Date,Sub-Category,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,Summary
9,Printers,3722.0,4750.0,3274.0,1055.0,2335.0,4349.0,5159.0,6290.0,3617.0,10867.0,2035.0,10799.0,58252.0
1,Bookcases,3374.0,1455.0,NaN,1489.0,8004.0,2989.0,3878.0,5274.0,2773.0,15564.0,5507.0,6554.0,56861.0
10,Saree,6483.0,6885.0,4445.0,1112.0,3300.0,6433.0,6389.0,4991.0,1403.0,3914.0,2128.0,6028.0,53511.0
8,Phones,5609.0,3348.0,3768.0,4932.0,2194.0,413.0,3928.0,3365.0,7564.0,3475.0,4688.0,2835.0,46119.0
2,Chairs,2251.0,3404.0,1415.0,840.0,570.0,1746.0,1854.0,5930.0,2765.0,3923.0,1764.0,7760.0,34222.0
16,Trousers,3659.0,1198.0,1901.0,327.0,5891.0,2908.0,642.0,3780.0,2638.0,742.0,833.0,5520.0,30039.0
0,Accessories,68.0,850.0,349.0,187.0,759.0,1470.0,1339.0,1515.0,2588.0,4871.0,3967.0,3765.0,21728.0
13,Stole,1056.0,335.0,680.0,482.0,1129.0,328.0,1892.0,2477.0,1396.0,3228.0,1759.0,3784.0,18546.0
5,Hankerchief,1116.0,299.0,413.0,202.0,345.0,428.0,1720.0,2444.0,1068.0,2242.0,1862.0,2469.0,14608.0
4,Furnishings,1132.0,751.0,673.0,1154.0,964.0,355.0,402.0,1974.0,542.0,1770.0,2883.0,884.0,13484.0


### Creating a column with coefficient of variation of sales. It gives information about sale stability (or volatility)

In [585]:
ordersubb['variation'] = 0
ordersubb['variation'] = ordersubb.iloc[:,1:13].apply(lambda x: np.std(x)/np.mean(x)*100, axis=1)

In [586]:
ordersubb.sort_values('variation')

Date,Sub-Category,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,Summary,variation
8,Phones,5609.0,3348.0,3768.0,4932.0,2194.0,413.0,3928.0,3365.0,7564.0,3475.0,4688.0,2835.0,46119.0,44.428426
10,Saree,6483.0,6885.0,4445.0,1112.0,3300.0,6433.0,6389.0,4991.0,1403.0,3914.0,2128.0,6028.0,53511.0,44.833779
12,Skirt,68.0,277.0,65.0,92.0,178.0,41.0,84.0,201.0,174.0,317.0,220.0,229.0,1946.0,53.667900
9,Printers,3722.0,4750.0,3274.0,1055.0,2335.0,4349.0,5159.0,6290.0,3617.0,10867.0,2035.0,10799.0,58252.0,61.786910
4,Furnishings,1132.0,751.0,673.0,1154.0,964.0,355.0,402.0,1974.0,542.0,1770.0,2883.0,884.0,13484.0,63.512921
5,Hankerchief,1116.0,299.0,413.0,202.0,345.0,428.0,1720.0,2444.0,1068.0,2242.0,1862.0,2469.0,14608.0,70.055361
13,Stole,1056.0,335.0,680.0,482.0,1129.0,328.0,1892.0,2477.0,1396.0,3228.0,1759.0,3784.0,18546.0,70.232379
11,Shirt,675.0,94.0,777.0,621.0,203.0,61.0,205.0,503.0,868.0,1062.0,1682.0,804.0,7555.0,70.945913
2,Chairs,2251.0,3404.0,1415.0,840.0,570.0,1746.0,1854.0,5930.0,2765.0,3923.0,1764.0,7760.0,34222.0,71.652783
16,Trousers,3659.0,1198.0,1901.0,327.0,5891.0,2908.0,642.0,3780.0,2638.0,742.0,833.0,5520.0,30039.0,72.828930


### Sales sub-categories summary with defining volatility by color

In [587]:
fig = px.bar(ordersubb, y='Summary', x='Sub-Category', title='Sales by sub-category', color='variation')
fig.show()

### Monthly sub-catogory sales visualization

In [588]:
import plotly.graph_objects as go
fig = go.Figure()
for i in range(ordersubb['Sub-Category'].nunique()):
    x = ordersubb.iloc[:, 1:13].columns
    fig.add_trace(go.Scatter(x = x, y = ordersubb.iloc[i, 1:13], mode = 'lines+markers', name = ordersubb['Sub-Category'].unique()[i]))
    i+=1  
fig.update_layout(title = 'Monthly Sub-categories sales')
fig.show()

### Defining XYZ categories. In classic X is for sales with volatility 5-15%, Y for volatility 15-50% and Z is for >50%. But here we have the minimum 44%, so assign categories based on this minimum: <50% is for X, 50-70% is for Y and >71% is for Z.

In [589]:
ordersubb['XYZ'] = 0
ordersubb.loc[(ordersubb['variation'] <= 50), 'XYZ'] = 'X'
ordersubb.loc[(ordersubb['variation'] <= 71)&(ordersubb['variation'] >= 50), 'XYZ'] = 'Y'
ordersubb.loc[(ordersubb['variation'] >= 71), 'XYZ'] = 'Z'
ordersubb

Date,Sub-Category,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,Summary,variation,XYZ
9,Printers,3722.0,4750.0,3274.0,1055.0,2335.0,4349.0,5159.0,6290.0,3617.0,10867.0,2035.0,10799.0,58252.0,61.786910,Y
1,Bookcases,3374.0,1455.0,NaN,1489.0,8004.0,2989.0,3878.0,5274.0,2773.0,15564.0,5507.0,6554.0,56861.0,73.977380,Z
10,Saree,6483.0,6885.0,4445.0,1112.0,3300.0,6433.0,6389.0,4991.0,1403.0,3914.0,2128.0,6028.0,53511.0,44.833779,X
8,Phones,5609.0,3348.0,3768.0,4932.0,2194.0,413.0,3928.0,3365.0,7564.0,3475.0,4688.0,2835.0,46119.0,44.428426,X
2,Chairs,2251.0,3404.0,1415.0,840.0,570.0,1746.0,1854.0,5930.0,2765.0,3923.0,1764.0,7760.0,34222.0,71.652783,Z
16,Trousers,3659.0,1198.0,1901.0,327.0,5891.0,2908.0,642.0,3780.0,2638.0,742.0,833.0,5520.0,30039.0,72.828930,Z
0,Accessories,68.0,850.0,349.0,187.0,759.0,1470.0,1339.0,1515.0,2588.0,4871.0,3967.0,3765.0,21728.0,85.415898,Z
13,Stole,1056.0,335.0,680.0,482.0,1129.0,328.0,1892.0,2477.0,1396.0,3228.0,1759.0,3784.0,18546.0,70.232379,Y
5,Hankerchief,1116.0,299.0,413.0,202.0,345.0,428.0,1720.0,2444.0,1068.0,2242.0,1862.0,2469.0,14608.0,70.055361,Y
4,Furnishings,1132.0,751.0,673.0,1154.0,964.0,355.0,402.0,1974.0,542.0,1770.0,2883.0,884.0,13484.0,63.512921,Y


### Joining results of ABC and XYZ

In [590]:
subb = orderssub[['Sub-Category','ABC']].merge(ordersubb[['Sub-Category','XYZ']], right_on = 'Sub-Category', left_on = 'Sub-Category', how = 'inner')
subb

Date,Sub-Category,ABC,XYZ
0,Printers,A,Y
1,Bookcases,A,Z
2,Accessories,A,Z
3,Trousers,A,Z
4,Stole,A,Y
5,Phones,A,X
6,Hankerchief,A,Y
7,T-shirt,B,Z
8,Shirt,B,Y
9,Furnishings,B,Y


## Result

In [591]:
table = subb.groupby(['ABC','XYZ'])['Sub-Category'].unique().unstack(level=-1)
table

XYZ,X,Y,Z
ABC,,,
A,[Phones],"[Printers, Stole, Hankerchief]","[Bookcases, Accessories, Trousers]"
B,NaN,"[Shirt, Furnishings]",[T-shirt]
C,[Saree],[Skirt],"[Chairs, Leggings, Kurti]"
